In [22]:
%load_ext autoreload
%autoreload 2
import re
import pandas as pd

from courtscraper.data_utils.consts import DOCCS_FOIL_TXT_PATH
from courtscraper.data_utils.consts import DOCCS_FOIL_TXT_PATH, DOCCS_FOIL_XLSX, TRACKER_PATH, \
    IGNORE, ETHNICITIES, CRIMES, COUNTIES
from courtscraper.ny.doccs_foil_text_to_xlsx import gen_xlsx

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
doccs_df = pd.read_excel(DOCCS_FOIL_XLSX)

In [54]:
def gen_convictions_df():
    convictions = []
    crimes = ['Most Serious Crime', 'Second Crime', 'Third Crime']
    counties = ['County of Indictment 1', 'County of Indictment 2', 'County of Indictment 3']
    for idx, fieldname in enumerate(crimes):
        for _, line in doccs_df.iterrows():
            if line[fieldname] == 'MURDER 2ND':
                convictions.append([
                    line['DIN'],
                    line['Name'],
                    line['Date of Birth'],
                    line['Ethnicity'],
                    line[fieldname],
                    line[counties[idx]],
                    line['Min Prison Term in Months']
                ])
    convs = pd.DataFrame(convictions)
    convs.columns = ['DIN', 'Name', 'DOB', 'Race', 'Crime', 'County', 'Min Term (Months)']

    convs.Name = convs.Name.apply(lambda x: x.title())
    convs.Race = convs.Race.apply(lambda x: x.title())
    convs.County = convs.County.apply(lambda x: x.title())
    convs.County = convs.County.apply(lambda x: 'St. Lawrence' if x == 'St Lawrence' else x)
    convs['YOB'] = convs.DOB.apply(lambda x: int(str(x)[:4]))
    convs.Race = convs.Race.apply(lambda x: x if x in ['Black', 'White', 'Hispanic', 'Asian'] else 'Other-Unknown')
    convs = convs.drop_duplicates()
    return convs

convs = gen_convictions_df()

In [55]:
# from collections import Counter
# Counter(convs['DOB'])

# # gen_xlsx('15A0634')
# # convs[convs['County'].apply(lambda x: type(x) != str)]
# doccs_df


/Users/eli/projects/felony-murder/.venv/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [71]:


tracker = pd.read_excel(TRACKER_PATH, sheet_name=1)
tracker['AutoName'] = tracker['Name']


for _, row in tracker.iterrows():
    if row['AutoName'] and str(row['AutoName']) != 'nan':
        continue
    if row['Crime Year'] != 'Unknown' and row['Crime Year'] >= 2020:
        continue
    match_convs = convs[convs['County'].apply(lambda x: x == row['Disposition County'])]
    min_term = row['Min Prison Term in Months']
    if str(min_term) in ['nan', 'Unknown']:
        match_convs = match_convs[
            match_convs['Min Term (Months)'].apply(
                lambda x: str(x) == 'nan')]
    else:
        match_convs = match_convs[
            match_convs['Min Term (Months)'].apply(
                lambda x: str(x) != 'nan' and int(x) == int(min_term))]

    if match_convs.empty:
        continue
    match_convs = match_convs[
        match_convs['Race'].apply(
            lambda x: x == row['Race/Ethnicity of Arrestee'])]
    if match_convs.empty:
        continue
    if row['Crime Year'] != 'Unknown':
        potential_yobs = [(row['Crime Year'] - row['Age at Crime']),
                         (row['Crime Year'] - row['Age at Crime']) - 1]
        match_convs = match_convs[
            match_convs['YOB'].apply(
                lambda x: x in potential_yobs)]
    if match_convs.empty:
        continue
    match_convs = match_convs[match_convs['DIN'].apply(
            lambda x: int(x[:2]) >= int(str(row['Arrest Year'])[2:])
        )]
    
    if not match_convs.empty:
        print('\n\n----------------------')
        print(f"Original Number: {row['Original Number']} Crime Year: {row['Crime Year']} Age at Arrest: {row['Age at Crime']}")
        print(f"\t Arrest Year: {row['Arrest Year']}")
        print(match_convs[['DIN', 'Name', 'DOB', 'Race', 'County', 'Min Term (Months)']])
        
        



----------------------
Original Number: 28 Crime Year: 2005 Age at Arrest: 19
	 Arrest Year: 2005
          DIN           Name       DOB      Race County  Min Term (Months)
200   09A1668  Carrasco Jose  19850312  Hispanic  Bronx              300.0
904   14A1473   Garcia Pablo  19861203  Hispanic  Bronx              300.0
1488  18A0295  Dejesus Jason  19860919  Hispanic  Bronx              300.0
1812  12A2099   Colon Paul J  19850813  Hispanic  Bronx              300.0


----------------------
Original Number: 202 Crime Year: 2005 Age at Arrest: 32
	 Arrest Year: 2005
         DIN           Name       DOB      Race County  Min Term (Months)
254  09A5681  Matias Miguel  19730626  Hispanic  Bronx              300.0
699  12A5300    Coello Eddy  19720914  Hispanic  Bronx              300.0


----------------------
Original Number: 400 Crime Year: 2006 Age at Arrest: 19
	 Arrest Year: 2006
         DIN             Name       DOB   Race County  Min Term (Months)
124  08B0274    Miller Jerem



----------------------
Original Number: 46 Crime Year: 2006 Age at Arrest: 50
	 Arrest Year: 2006
        DIN             Name       DOB   Race County  Min Term (Months)
26  08A1831  Scrubb Anderson  19550923  Black  Kings              300.0


----------------------
Original Number: 83 Crime Year: 2007 Age at Arrest: 18
	 Arrest Year: 2007
         DIN           Name       DOB   Race County  Min Term (Months)
250  09A5493  Smith Maurice  19891129  Black  Kings              240.0
413  10A5450   Moore Horace  19880228  Black  Kings              240.0
842  13A5667    Deane Akeem  19890708  Black  Kings              240.0


----------------------
Original Number: 149 Crime Year: 2007 Age at Arrest: 29
	 Arrest Year: 2007
          DIN               Name       DOB   Race County  Min Term (Months)
220   09A2998      Bramble Anton  19780722  Black  Kings              300.0
239   09A4055      Parker Keenan  19780104  Black  Kings              300.0
416   10A5643       Brummel Teon  19770409 



----------------------
Original Number: 37 Crime Year: 2005 Age at Arrest: 25
	 Arrest Year: 2006
          DIN            Name       DOB   Race  County  Min Term (Months)
1832  14A4437  Fuller Darrell  19790922  Black  Nassau                NaN


----------------------
Original Number: 369 Crime Year: 2010 Age at Arrest: 30
	 Arrest Year: 2010
          DIN            Name       DOB   Race  County  Min Term (Months)
1832  14A4437  Fuller Darrell  19790922  Black  Nassau                NaN


----------------------
Original Number: 661 Crime Year: 2013 Age at Arrest: 34
	 Arrest Year: 2013
          DIN            Name       DOB   Race  County  Min Term (Months)
1832  14A4437  Fuller Darrell  19790922  Black  Nassau                NaN


----------------------
Original Number: 698 Crime Year: 2009 Age at Arrest: 29
	 Arrest Year: 2011
          DIN            Name       DOB   Race  County  Min Term (Months)
1832  14A4437  Fuller Darrell  19790922  Black  Nassau                NaN


---



----------------------
Original Number: 1097 Crime Year: 2016 Age at Arrest: 27
	 Arrest Year: 2016
          DIN            Name       DOB   Race  County  Min Term (Months)
1709  19A4104  Jones Sahquane  19890524  Black  Queens              300.0


----------------------
Original Number: 1099 Crime Year: 2016 Age at Arrest: 36
	 Arrest Year: 2016
          DIN         Name       DOB      Race  County  Min Term (Months)
1858  18A2375  Morel Oscar  19800818  Hispanic  Queens                NaN


----------------------
Original Number: 1135 Crime Year: 2018 Age at Arrest: 21
	 Arrest Year: 2018
          DIN            Name       DOB   Race  County  Min Term (Months)
1863  19A1626  Lewis Chanel R  19960502  Black  Queens                NaN


----------------------
Original Number: 1184 Crime Year: 2013 Age at Arrest: 16
	 Arrest Year: 2013
          DIN            Name       DOB   Race  County  Min Term (Months)
1863  19A1626  Lewis Chanel R  19960502  Black  Queens                NaN
